## Allocate resources in remote container

In [ ]:
from kubeflow import fairing
from kubeflow.fairing import TrainJob
from kubeflow.fairing.backends import KubeflowAWSBackend
from kubeflow.fairing.kubernetes.utils import get_resource_mutator

In [ ]:
import multiprocessing
import os
import sys

def train():
    print("CPU count: {}".format(multiprocessing.cpu_count()))
    print("Memory: {}".format(os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')/(1024.**3)))

train()

In [ ]:
AWS_ACCOUNT_ID=fairing.cloud.aws.guess_account_id()
AWS_REGION='us-west-2'
DOCKER_REGISTRY = '{}.dkr.ecr.{}.amazonaws.com'.format(AWS_ACCOUNT_ID, AWS_REGION)
PY_VERSION = ".".join([str(x) for x in sys.version_info[0:3]])
BASE_IMAGE = '{}/python:{}'.format(DOCKER_REGISTRY, PY_VERSION)
# TODO: bug to fix. use tensorflow image temporarily
BASE_IMAGE = 'tensorflow/tensorflow:1.14.0-py3'

job = TrainJob(train, base_docker_image=BASE_IMAGE, docker_registry=DOCKER_REGISTRY,  backend=KubeflowAWSBackend(), 
              pod_spec_mutators=[get_resource_mutator(cpu=3, memory=4)])
job.submit()